In [1]:
import torch
from torch.utils.data import DataLoader
from functools import partial
import transformers
from tqdm import tqdm
import pickle

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from models import build_model

/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.0+cu118 with CUDA 1108 (you have 2.5.1+cu124)
    Python  3.10.11 (you have 3.10.15)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
/home/jovyan/.mlspace/envs/

Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'


/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
import json

with open('val_qa_msrvtt.json') as f:
    dataset = json.load(f)
    print(len(dataset))

12278


In [10]:
dataset[1:10]

[{'answer': 'coversation',
  'category_id': 14,
  'id': 158582,
  'question': 'what is a family having?',
  'video_id': 6513},
 {'answer': 'music',
  'category_id': 14,
  'id': 158583,
  'question': 'what plays?',
  'video_id': 6513},
 {'answer': 'music',
  'category_id': 14,
  'id': 158584,
  'question': 'what is playing?',
  'video_id': 6513},
 {'answer': 'couch',
  'category_id': 14,
  'id': 158585,
  'question': 'what does the girl sing on?',
  'video_id': 6513},
 {'answer': 'feature',
  'category_id': 14,
  'id': 158586,
  'question': 'what is a music video doing?',
  'video_id': 6513},
 {'answer': 'sit',
  'category_id': 14,
  'id': 158587,
  'question': 'what are people doing?',
  'video_id': 6513},
 {'answer': 'sit',
  'category_id': 14,
  'id': 158588,
  'question': 'what are three people doing?',
  'video_id': 6513},
 {'answer': 'sit',
  'category_id': 14,
  'id': 158589,
  'question': 'what are three young people doing?',
  'video_id': 6513},
 {'answer': 'talk',
  'category_

In [ ]:
shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/eval/YouTubeClips/W8yjP35v1EU.avi

In [5]:
from collections import defaultdict

id2video_mapper = defaultdict()

with open("youtube_mapping.txt") as file:
    lines = [line.rstrip() for line in file]

for line in lines:
    key, value = line.split()
    value = value.split('vid')[1]
    id2video_mapper[value] = key


In [7]:

class msrvttDataset(torch.utils.data.Dataset):

    def __init__(self, module_path, dataset, mapper, task_type = 'qa', question_prompt = " Answer the question using a single word or a short phrase with multiple words."):
        
        self.data_list = []
        self.question_prompt = question_prompt

        self.movie_dir = module_path

        for elem in dataset:
            full_video_path = module_path + '/eval/YouTubeClips' +'/' + mapper[str(elem['video_id'])] +".avi"
            self.data_list.append({'full_video_path':full_video_path, 'question':elem['question'], 'answer':elem['answer']})
            

        
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        data_item = self.data_list[idx]
        video_path = data_item['full_video_path']
        
        if not os.path.exists(video_path):
            print (video_path)
            print(f"Warning: Video file not found at {video_path}, skipping this item.")
            return None  
    

        question = data_item['question']
        answer = data_item['answer']
    
        return {
            'question': question.capitalize() + self.question_prompt,
            'video_path': video_path,
            #'pixel_values': pixel_values,
            'answer': answer.capitalize(),
        }
    

In [8]:
msrvtt = msrvttDataset(module_path, dataset, id2video_mapper, task_type = 'qa') 

KeyError: '6513'

## Inference

In [8]:
import os
import sys
import torch
import numpy as np
import random
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from models import build_model

model_path = "/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft"#"/home/jinyang06/models/VideoLaVIT-v1/language_model_sft"
model_dtype='bf16'

max_video_clips = 16
device_id = 0
torch.cuda.set_device(device_id)
device = torch.device('cuda')

seed = 42
#torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# For Multi-Modal Understanding
runner = build_model(model_path=model_path, model_dtype=model_dtype, understanding=True, 
        device_id=device_id, use_xformers=False, max_video_clips=max_video_clips,)

Loading Video LaVIT Model Weight from /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft, model precision: bf16
Not used {}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft were not used when initializing VideoLaVITLlamaForCausalLM: ['model.motion_tokenizer.quantize.embedding.initted', 'model.motion_tokenizer.quantize.embedding.embed_avg', 'model.motion_tokenizer.quantize.embedding.cluster_size', 'model.motion_tokenizer.quantize.cluster_size']
- This IS expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The Visual Vocab Size is 16384
The llama tokenizer vocab size is 32000
The maximal clip number is 16


In [14]:
video_path = '/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/eval/YouTubeClips/bQJQGoJF7_k_162_169.avi'
prompt = "Who pours marinade in a bag of chicken? Answer the question using a single word or phrase."

output = runner({"video": video_path, "text_input": prompt}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
print(output)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
Man


In [10]:
tokenizer = transformers.LlamaTokenizer.from_pretrained(model_path, use_fast=False, padding_side='left')
tokenizer.pad_token = tokenizer.unk_token

In [12]:
def collate_fn(batches, tokenizer):
    
    questions = [_['question'] for _ in batches]
    video_path = [_['video_path'] for _ in batches]
    answer = [_['answer'] for _ in batches]
    
    return questions, video_path, answer

dataloader = torch.utils.data.DataLoader(
        dataset=msvdq,
        batch_size=1,
        num_workers=1,
        pin_memory=True,
        drop_last=False,
        collate_fn=partial(collate_fn, tokenizer=tokenizer)
    )

iterator = iter(dataloader)
first_batch = next(iterator)


question = first_batch[0]
video_path = first_batch[1]
answer = first_batch[2]


print(question)
print(answer)
print(video_path)


first_batch = next(iterator)


question = first_batch[0]
video_path = first_batch[1]
answer = first_batch[2]


print(question)
print(answer)
print(video_path)


first_batch = next(iterator)


question = first_batch[0]
video_path = first_batch[1]
answer = first_batch[2]


print(question)
print(answer)
print(video_path)


['Who pours liquid from a plastic container into a ziploc bag containing meat pieces? Answer the question using a single word or a short phrase with multiple words.']
['Someone']
['/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/eval/YouTubeClips/bQJQGoJF7_k_162_169.avi']
['Who pours a seasoning liquid from a plastic container over chicken pieces placed in a plastic pouch? Answer the question using a single word or a short phrase with multiple words.']
['Man']
['/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/eval/YouTubeClips/bQJQGoJF7_k_162_169.avi']
['Who pours marinade in a bag of chicken? Answer the question using a single word or a short phrase with multiple words.']
['Person']
['/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/eval/YouTubeClips/bQJQGoJF7_k_162_169.avi']


In [ ]:
y_pred = []
y_real = []

progress_bar = tqdm(
        dataloader, total=len(dataloader), desc=f"Epoch 1"
    )

output_list = []

for step, batch in enumerate(progress_bar, start=1):
    sample_set = {}
    question = batch[0]
    video_path = batch[1]
    answer = batch[2]


    sample_set['q'] = question
    sample_set['real'] = answer[0].strip()

    # print(pixel_values.size())
    # print(question)
    # print(answer)
    # print(num_patches_list)
    # print(task_type)
    
    # with autocast():
        # 执行推理
    outputs = runner({"video": video_path[0], "text_input": question[0]}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]

    sample_set['a'] = outputs.strip()

    y_pred.append(outputs.strip())
    
    y_real.append(answer[0].strip())

    output_list.append(sample_set)

    sample_set['a'] = outputs.strip()
    if (step%100 == 0):
        #with open('y_pred_msvdqa1.pkl', 'wb') as f:
            #pickle.dump(y_pred, f)
        #with open('y_real_msvdqa1.pkl', 'wb') as f:
            #pickle.dump(y_real, f)

        with open("sample_set.json", 'w') as file:
            json.dump(output_list, file)

        print("="*20 + "question" + "="*20)
        print (question[0])
        print("="*20 + "output" + "="*20)
        print(outputs)
        print("="*20 + "real answers" + "="*20)
        print(answer[0], flush = True)

Epoch 1:  70%|███████   | 4499/6415 [1:46:04<27:06,  1.18it/s]

====================question====================
Who is pouring olive oil into a pan? Answer the question using a single word or a short phrase with multiple words.
====================output====================
Chef
====================real answers====================
Someone


Epoch 1:  75%|███████▍  | 4799/6415 [1:50:46<24:42,  1.09it/s]

====================question====================
What is a woman seated by a lake pulled at by a black gloved hand? Answer the question using a single word or a short phrase with multiple words.
====================output====================
Grabbed
====================real answers====================
Ankle


Epoch 1:  76%|███████▋  | 4899/6415 [1:52:09<24:08,  1.05it/s]

...

Epoch 1:  95%|█████████▌| 6099/6415 [2:19:25<13:41,  2.60s/it]

====================question====================
What is someone kneeding? Answer the question using a single word or a short phrase with multiple words.
====================output====================
Dough
====================real answers====================
Dough


Epoch 1:  97%|█████████▋| 6199/6415 [2:21:11<02:40,  1.35it/s]

====================question====================
What does a man cut a chicken with? Answer the question using a single word or a short phrase with multiple words.
====================output====================
The man cuts a chicken on the ground
====================real answers====================
Axe


Epoch 1: 100%|██████████| 6415/6415 [2:25:12<00:00,  1.36s/it]


In [ ]:
y_pred[:30]

['Man',
 'Man',
 'Man',
 'Bbq sauce',
 'Man',
 'The person pouring marinade into a bag on a kitchen counter is a woman.',
 'No one',
 'Man',
 'Man',
 'Chef',
 'Man',
 'Sauce',
 'Man',
 'Man',
 'Marinade',
 'Man',
 'Sauce',
 'Meat',
 'Clear plastic bag',
 'Plastic container',
 'Marinade',
 'Chicken',
 'Bag',
 'Dressing',
 'Olive oil',
 'Sauce',
 'Spices',
 'Sauce',
 'Chicken',
 'Styrofoam container']

In [ ]:
y_real[:30]

['Someone',
 'Man',
 'Person',
 'Man',
 'Man',
 'Man',
 'Man',
 'Cook',
 'Man',
 'Person',
 'Person',
 'Seasoning',
 'Man',
 'Man',
 'Sauce',
 'Man',
 'Marinade',
 'Chicken',
 'Bag',
 'Bag',
 'Bowl',
 'Chicken',
 'Bag',
 'Marinade',
 'Sauce',
 'Marinade',
 'Marinade',
 'Sauce',
 'Meat',
 'Bag']

In [15]:
result = [1 if i == j else 0 for i, j in zip(y_real, y_pred)]

print("VideoLAVIT_MSVDQA, accuracy is: " + str(sum(result) / len(result)))

VideoLAVIT_MSVDQA, accuracy is: 0.21818181818181817


## Evaluation by Gpt-3

In [ ]:
import openai

# Set your OpenAI API key
openai.api_key = "YOUR_API_KEY"

# Define the test cases
test_cases = [
    {"input": "What is the capital of France?", "expected_output": "Paris"},
    {"input": "What is 2 + 2?", "expected_output": "4"},
    {"input": "Explain photosynthesis briefly.", "expected_output": "Photosynthesis is the process by which plants convert sunlight into energy."},
]

# Validate the model
for test in test_cases:
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": test["input"]}]
    )
    output = response['choices'][0]['message']['content'].strip()
    print(f"Input: {test['input']}")
    print(f"Expected: {test['expected_output']}")
    print(f"Output: {output}")
    print(f"Match: {output == test['expected_output']}")
    print("-" * 30)